In [2]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from keras.layers import Input, Dense
from keras.losses import MeanSquaredError
from keras import Model
import pandas as pd
import tensorflow as tf

/home/yu/kaggle-tweet-sentiment/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-15 18:09:14.530921: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-15 18:09:14.531898: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-15 18:09:14.536085: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-15 18:09:14.547126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting 

In [3]:
train = pd.read_csv("data1/train.csv").dropna()
submission = pd.read_csv("data1/test.csv").dropna()

def append_sentiment(df):
    return df.sentiment.str.strip() + " " + df.text.str.strip()

train["text"] = append_sentiment(train)
train["selected_from"] = train.apply(lambda x: x.text.find(x.selected_text), axis=1)
train["selected_to"] = train.apply(lambda x: x.selected_from + len(x.selected_text), axis=1)
x_train = train.text
y_train = train[["selected_from", "selected_to"]]
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.15)

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
embedding = tokenizer(x_train.to_list(), padding="max_length", max_length=256, truncation=True)

In [9]:
inputs = Input(shape=(256,))
outputs = Dense(1, activation="relu")(inputs)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss=MeanSquaredError, metrics=["accuracy"])
model.summary()

2025-01-15 16:49:11.453494: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 257 (1.00 KB)

 Trainable params: 257 (1.00 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
history = model.fit(tf.constant(embedding["input_ids"]), y_train, batch_size=64, epochs=10)

Epoch 1/10
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 7.6288e-05 - loss: 3129.6018 
Epoch 2/10
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.4616e-04 - loss: 3111.9370  
Epoch 3/10
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 8.8511e-05 - loss: 3112.7156  
Epoch 4/10
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.1439e-04 - loss: 3103.8938 
Epoch 5/10
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.4990e-04 - loss: 3115.2754  
Epoch 6/10
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.2410e-04 - loss: 3081.9329  
Epoch 7/10
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 8.4931e-05 - loss: 3127.6072  
Epoch 8/10
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.1141e-04 - loss: 3150.7041  
Epoch 9/10
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 7.1698e-05 - loss: 3138.6853  
Epoch 10/10
365/365 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 5.6792e-05 - loss: 3135.3794


In [23]:
test_embedding = tokenizer(x_test.to_list(), padding="max_length", max_length=256, truncation=True)["input_ids"]
model.evaluate(tf.constant(test_embedding), y_test, verbose=2)

129/129 - 0s - 1ms/step - accuracy: 0.0000e+00 - loss: 3170.0251


[3170.025146484375, 0.0]